In [105]:
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
import os
from tqdm import tqdm_notebook as tqdm
import time
from lxml import etree
from sklearn.metrics import r2_score
from datetime import timedelta
import numpy as np

In [106]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def pretty_print_result(search_result, fields=None):
    if fields is None:
        fields = []
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')


def get_score(search_result):
    res = []
    for hit in search_result['hits']['hits']:
        res.append((hit["_id"], hit["_score"]))
    res.sort(key = lambda x: x[1], reverse = True)
    return res


class Index:
    def __init__(self, index, settings):
        self.index_name = index
        self.settings = settings
        self.es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360}])
        if self.es.indices.exists(index=index):
            self.es.indices.delete(index=index)
        self.es.indices.create(index=index, body=settings)

    def es_actions_generator(self, path_to_docs):
        for doc_name in tqdm(os.listdir(path_to_docs)):
            with open(f"{path_to_docs}/{doc_name}", "r", encoding="utf-8") as inf:
                doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
                doc = json.load(inf)           
            yield create_es_action(self.index_name, doc_id, doc)


    def add_documents(self, path_to_docs):
        try:
            for ok, result in parallel_bulk(self.es, self.es_actions_generator(path_to_docs), queue_size=4, thread_count=4,
                                        chunk_size=1000):
                  if not ok:
                     print(result)
        except Exception as e: 
            print(e)

    def get_doc_by_id(self, doc_id):
        return self.es.get(index=self.index_name, id=doc_id)['_source']

    def search(self, query, *args):
        return self.es.search(index=self.index_name, body=query, size=20)
        # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20



In [107]:
settings_1 = {
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            }
        }
    }
}

In [135]:
settings = {
        'mappings': {
            'properties': {
                'text': {
                    'type': 'text',
                    'analyzer': 'russian_complex',
                    'search_analyzer': 'russian_complex'
                }
            }
            
        },
        "settings": {
        "analysis" : {
            "analyzer" : {
                "my_analyzer" : {
                    "tokenizer" : "standard",
                    "filter" : ["lowercase", "russian_snow", "english_snow"]
                },
                'russian_complex': {
                    'char_filter': [
                        'yont'
                    ],
                    'tokenizer': 'word_longer_2',
                    'filter': [
                        'lowercase',
                        'russian_snow'
                    ]
                }
            },
            'char_filter': {
                'yont': {
                    'type': 'mapping',
                    'mappings': [
                        'ё => е'
                    ]
                }
            },
            'tokenizer': {
                'word_longer_2': {
                    'type': 'pattern',
                    'pattern': '[a-zA-Z_0-9\u0400-\u04FF]{2,}',
                    'group': 0
                },
                'white_20': {
                    'type': 'whitespace',
                    'max_token_length': 5
                }
            },
            "filter" : {
                "russian_snow" : {
                    "type" : "snowball",
                    "language" : "Russian"
                },
                "english_snow" : {
                    "type" : "snowball",
                    "language" : "English"
                }
            }
        }
    }
}

In [136]:
index = Index("docs", settings_1)

In [110]:
start = time.time()
index.add_documents("res/json")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:03:17.092622


In [137]:
stem_index = Index('stem_docs', settings)

In [138]:
start = time.time()
stem_index.add_documents("res/json")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:04:46.952495


In [139]:
stem_index.get_doc_by_id('1000039')

{'text': 'сервисы | о компании | партнерам Описание Рубрикатор Мобильный переводчик SMS-ИНФО - услуга, позволяющая с помощью SMS-запроса оперативно получать информацию различного \nхарактера в текстовом виде. Информация SMS-ИНФО мобильна, доступна 24 часа в сутки на всей территории \nзоны действия сети VELCOM. Ответ на запрос приходит оперативно - в течение нескольких секунд. Для получения информации необходимо отправить SMS-запрос (латиницей или кириллицей) на номер 511. Кроме того, можно сформировать \nтакой SMS-запрос с помощью SIM-карты нового образца с VELCOM-меню. Таблица транслитерации (соответствия \nрусских букв латинским) представлена в разделе Инструкция по SMS-ИНФО. Полный список рубрик SMS-ИНФО можно получить, отправив SMS-запрос HELP на номер 511. Стоимость одного запроса на номер 511 составляет 110 рублей без учета НДС и налога с продаж; для абонентов. Подробнее об услуге SMS-ИНФО читайте в разделе Инструкция по SMS-ИНФО. Внимание! Информацию можно получать и в виде MMS-

In [111]:
class Query:
    def __init__(self, task_id, query, relevant_docs):
        self.task_id = task_id
        self.query = query
        self.relevant_docs = relevant_docs

def json_query(query):
    return {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'text': query.query
                    }
                }
            ]
        }
    }
}
        


In [112]:
def pagerank_query(query):
    return  {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'text': query.query
                    }
                },
                {
                    'rank_feature': {
                        'field': 'pagerank',
                        'saturation': {
                            'pivot': 10
                        },
                        'boost': '5.0'
                    }
                }
            ]
        }
    }
}

In [119]:
class SearchQualityChecker:
    def __init__(self, queries, index):
        self.queries = queries
        self.index = index
        self.results = {}
        
    def get_results(self, get_query=json_query):
        r_precision_total = 0
        map_score_total = 0
        r_total = 0
        p_total = 0
        for q in tqdm(self.queries):
            res = self.index.search(get_query(q))
            print(q.task_id)
            pretty_print_result(res)
            scores = get_score(res)
            p_total += self.p(20, q, scores)
            r_total += self.r(20, q, scores)
            r_precision_total += self.r_precision(q, scores)
            map_score_total += self.map_score(q, scores, 20)
        Q = len(self.queries)
        return r_precision_total / Q, map_score_total / Q, p_total / Q, r_total / Q
    
    def r_precision(self, query, search_res_score):
        return self.r(len(query.relevant_docs), query, search_res_score)
    
    def map_score(self, query, search_res_score, n):
        m = 0
        for k in range(1, n):       
            m += self.p(k, query, search_res_score)
        R = len(query.relevant_docs)
        return m / n
    
    def p(self, k, query, search_res_score):
        r = 0
        for doc, _ in search_res_score[:k]:
            if doc in query.relevant_docs:
                r += 1
        return r / k
    
    def r(self, k, query, search_res_score):
        R = len(query.relevant_docs)
        r = 0
        for doc, _ in search_res_score[:k]:
            if doc in query.relevant_docs:
                r += 1
        return r / R if R != 0 else 0 if len(search_res_score) > 0 else 1


In [120]:
def get_relevance():
    res = {}
    xml_tree = etree.parse("data/or_relevant-minus_table.xml")
    root = xml_tree.getroot()
    for task in root.getchildren():
        relevant_docs = set()
        for document in task.getchildren():
            if document.get("relevance") == "vital":
                relevant_docs.add(document.get("id"))
        res[task.get("id")] = relevant_docs
    print(len(res))
    return res


def generate_queries_plain_texts():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in root.getchildren():
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    res.append(Query(task.get("id"), query_text.text, relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res

In [121]:
queries = generate_queries_plain_texts()
quality_checker = SearchQualityChecker(queries, index)
plain_text_res = quality_checker.get_results()

547
547


arw49633
Total documents: 1911
Doc 990390, score is 13.157524
Doc 993272, score is 13.157524
Doc 359461, score is 13.157524
Doc 360857, score is 12.175998
Doc 114021, score is 9.994495
Doc 863522, score is 9.937239
Doc 863494, score is 9.902861
Doc 599962, score is 9.885215
Doc 969889, score is 9.791648
Doc 802136, score is 9.790344
Doc 77084, score is 9.769339
Doc 89548, score is 9.744069
Doc 1319849, score is 9.6186695
Doc 623630, score is 9.608261
Doc 971553, score is 9.583519
Doc 1283004, score is 9.556349
Doc 86909, score is 9.48445
Doc 1427859, score is 9.48445
Doc 77545, score is 9.461813
Doc 658433, score is 9.452979
arw49662
Total documents: 0
arw49674
Total documents: 10000
Doc 74265, score is 20.83848
Doc 876069, score is 19.587797
Doc 1115273, score is 17.429165
Doc 19214, score is 17.099628
Doc 19995, score is 17.099628
Doc 1509861, score is 16.007002
Doc 74270, score is 15.681253
Doc 1360296, score is 15.483906
Doc 681222, score is 15.483906
Doc 878912, score is 14.794117

arw49936
Total documents: 10000
Doc 951580, score is 17.564886
Doc 886237, score is 17.314306
Doc 1499101, score is 17.257875
Doc 1263409, score is 17.193613
Doc 18647, score is 17.174175
Doc 1342194, score is 17.15792
Doc 1380429, score is 17.114546
Doc 696668, score is 17.114546
Doc 435356, score is 16.669022
Doc 1345166, score is 16.165646
Doc 886078, score is 16.121595
Doc 969907, score is 16.121471
Doc 1401098, score is 16.043303
Doc 1488655, score is 15.7847185
Doc 1435053, score is 15.529593
Doc 1045967, score is 15.359879
Doc 1181639, score is 15.280785
Doc 1218350, score is 15.240352
Doc 1429081, score is 15.116403
Doc 1267206, score is 14.867378
arw49938
Total documents: 10000
Doc 108087, score is 15.39679
Doc 490775, score is 13.221358
Doc 593314, score is 13.119261
Doc 1522926, score is 13.119261
Doc 432781, score is 12.770277
Doc 636717, score is 12.770277
Doc 853616, score is 12.292774
Doc 527481, score is 12.292774
Doc 618444, score is 12.039942
Doc 417346, score is 11.7

arw50191
Total documents: 7461
Doc 402431, score is 28.38454
Doc 614686, score is 26.936474
Doc 1353937, score is 25.463774
Doc 694703, score is 25.209534
Doc 1252848, score is 24.926878
Doc 300630, score is 23.232151
Doc 1039968, score is 22.828705
Doc 1455592, score is 22.308514
Doc 1059765, score is 22.231182
Doc 624771, score is 22.02257
Doc 1138214, score is 21.67081
Doc 1455416, score is 21.42957
Doc 480187, score is 21.351116
Doc 254889, score is 21.259575
Doc 802169, score is 21.11596
Doc 1371639, score is 21.096764
Doc 1214163, score is 20.904005
Doc 13995, score is 20.760763
Doc 1345946, score is 20.195251
Doc 1387487, score is 20.13168
arw50197
Total documents: 10000
Doc 427952, score is 44.394287
Doc 77192, score is 38.25826
Doc 1286031, score is 36.802708
Doc 801663, score is 35.733795
Doc 1512211, score is 35.45205
Doc 1326805, score is 35.088627
Doc 48294, score is 34.84206
Doc 1259302, score is 34.795094
Doc 44902, score is 34.34107
Doc 401661, score is 33.87911
Doc 721

arw50498
Total documents: 99
Doc 1012024, score is 14.120238
Doc 845794, score is 14.095792
Doc 314169, score is 13.819838
Doc 703623, score is 13.486127
Doc 1139478, score is 13.393441
Doc 101476, score is 13.270543
Doc 376855, score is 13.166261
Doc 514284, score is 12.947877
Doc 427993, score is 12.722467
Doc 1378017, score is 12.722467
Doc 443498, score is 12.722467
Doc 482973, score is 12.695429
Doc 459625, score is 12.6533375
Doc 833497, score is 12.294401
Doc 680169, score is 12.068155
Doc 576081, score is 11.776447
Doc 5172, score is 11.774935
Doc 882083, score is 11.566755
Doc 949647, score is 11.566755
Doc 887228, score is 11.042431
arw50507
Total documents: 985
Doc 977476, score is 22.466837
Doc 944379, score is 20.615173
Doc 8972, score is 20.274193
Doc 1439417, score is 20.171535
Doc 443608, score is 19.73111
Doc 36886, score is 19.250051
Doc 822957, score is 19.250051
Doc 31717, score is 19.250051
Doc 350559, score is 18.779512
Doc 1266495, score is 18.693344
Doc 994663, 

...

arw57647
Total documents: 1652
Doc 1521661, score is 18.827793
Doc 532031, score is 18.049656
Doc 721856, score is 17.249317
Doc 1277069, score is 15.252264
Doc 1032461, score is 14.927492
Doc 1371901, score is 14.717339
Doc 118087, score is 14.594564
Doc 1309098, score is 14.505798
Doc 1313005, score is 14.408232
Doc 1100474, score is 13.934862
Doc 1197646, score is 13.741484
Doc 1113172, score is 13.741484
Doc 518060, score is 13.345091
Doc 412874, score is 13.070509
Doc 1373554, score is 12.831212
Doc 116997, score is 12.496597
Doc 1100205, score is 12.496597
Doc 373941, score is 11.955669
Doc 1024181, score is 11.955669
Doc 941790, score is 11.475455
arw57701
Total documents: 3653
Doc 472928, score is 32.596184
Doc 1404303, score is 29.969292
Doc 472913, score is 28.624008
Doc 1149318, score is 28.12512
Doc 1187965, score is 27.430988
Doc 933692, score is 26.337883
Doc 1000301, score is 26.039583
Doc 1361878, score is 24.916798
Doc 1292335, score is 24.879787
Doc 1518333, score is 

arw57881
Total documents: 10000
Doc 575214, score is 20.189432
Doc 965044, score is 19.294333
Doc 585578, score is 15.903329
Doc 105132, score is 12.282671
Doc 1430591, score is 11.899367
Doc 347330, score is 11.838234
Doc 962880, score is 11.655708
Doc 799700, score is 9.450956
Doc 1158877, score is 9.32595
Doc 1243202, score is 9.041545
Doc 1407793, score is 8.761847
Doc 1144476, score is 8.07221
Doc 1311572, score is 7.240443
Doc 1094376, score is 7.235602
Doc 311924, score is 7.1314797
Doc 1162458, score is 7.101149
Doc 76199, score is 7.074702
Doc 865655, score is 7.0254793
Doc 660609, score is 7.004841
Doc 1190789, score is 7.004841
arw57932
Total documents: 10000
Doc 880639, score is 19.948145
Doc 986290, score is 19.63608
Doc 906465, score is 19.34823
Doc 395954, score is 19.078842
Doc 446652, score is 17.954287
Doc 486491, score is 17.615168
Doc 1086269, score is 17.168764
Doc 49459, score is 17.122644
Doc 1378167, score is 16.943407
Doc 827938, score is 16.86401
Doc 1479444, 

arw58309
Total documents: 10000
Doc 893805, score is 29.099545
Doc 893800, score is 28.63488
Doc 573076, score is 27.136295
Doc 1272980, score is 26.227573
Doc 14202, score is 25.629623
Doc 1347725, score is 25.349981
Doc 963247, score is 25.271112
Doc 1085068, score is 24.866455
Doc 1287074, score is 24.617167
Doc 1221750, score is 24.447718
Doc 65978, score is 24.185253
Doc 539628, score is 24.12385
Doc 899211, score is 24.114517
Doc 1109251, score is 23.7959
Doc 903306, score is 23.693695
Doc 1043979, score is 23.564892
Doc 1282616, score is 23.563654
Doc 773606, score is 23.215055
Doc 1157976, score is 23.201338
Doc 87671, score is 22.980484
arw58328
Total documents: 10000
Doc 1181049, score is 14.501312
Doc 1069025, score is 14.300163
Doc 774457, score is 13.869848
Doc 678850, score is 13.771218
Doc 876651, score is 13.689619
Doc 964505, score is 13.689619
Doc 1079497, score is 13.689619
Doc 44756, score is 13.499004
Doc 420852, score is 13.374075
Doc 1079506, score is 13.374075
D

arw58656
Total documents: 10000
Doc 1011797, score is 18.854252
Doc 888082, score is 18.648142
Doc 885267, score is 15.682592
Doc 720243, score is 15.160116
Doc 622524, score is 14.612402
Doc 1160367, score is 14.23526
Doc 1215054, score is 13.591521
Doc 1009296, score is 8.266839
Doc 1409610, score is 0.30556768
Doc 1077248, score is 0.30437756
Doc 1459068, score is 0.30426902
Doc 1234647, score is 0.30392897
Doc 320801, score is 0.30387834
Doc 1153649, score is 0.30386227
Doc 1050943, score is 0.3037649
Doc 452748, score is 0.30376276
Doc 1268249, score is 0.3036822
Doc 357297, score is 0.30361316
Doc 1101758, score is 0.303599
Doc 374920, score is 0.30359265
arw58729
Total documents: 6684
Doc 1227722, score is 18.671474
Doc 1380591, score is 17.906292
Doc 1382034, score is 17.851414
Doc 1153147, score is 17.747498
Doc 1396046, score is 15.178157
Doc 1203374, score is 14.774771
Doc 1289039, score is 14.190643
Doc 378067, score is 13.354945
Doc 574281, score is 13.175722
Doc 513355, s

arw58970
Total documents: 10000
Doc 1262228, score is 13.346936
Doc 1473606, score is 12.404811
Doc 1421173, score is 12.29347
Doc 672364, score is 11.570872
Doc 905038, score is 11.05001
Doc 1077331, score is 10.870287
Doc 661559, score is 10.839827
Doc 39399, score is 10.818637
Doc 1516902, score is 10.383684
Doc 626613, score is 10.38235
Doc 72651, score is 10.38235
Doc 887844, score is 10.319556
Doc 1262447, score is 10.283515
Doc 527356, score is 10.258385
Doc 1166149, score is 10.1987915
Doc 62632, score is 10.079145
Doc 296316, score is 10.019127
Doc 1227069, score is 9.959688
Doc 431289, score is 9.848082
Doc 462175, score is 9.848082
arw58974
Total documents: 9360
Doc 1519881, score is 16.95644
Doc 1393897, score is 16.459774
Doc 917546, score is 16.390667
Doc 665645, score is 16.266926
Doc 1308001, score is 15.754616
Doc 819031, score is 15.653839
Doc 1501015, score is 15.599531
Doc 1358149, score is 15.543877
Doc 1201133, score is 15.543877
Doc 1380743, score is 15.543877
Do

In [42]:
plain_text_res

(0.18016356694188573,
 0.3444619661098512,
 0.3020109689213894,
 0.20969783400006586)

In [43]:
from pymystem3 import Mystem
from nltk.corpus import stopwords
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")
black_list = ["°", "№", "©", "...", "//", "://", "</", "\">", "=\"", "=\'", "\r", "\n", "\t"]
stem = Mystem()

def lemmatize(text):
    words = nltk.word_tokenize(text.lower())
    tokens = []
    for word in words:
        tokens.extend(stem.lemmatize(word))
    tokens = [token for token in tokens if token != " " and token.strip() not in punctuation \
              and token not in russian_stopwords and token not in english_stopwords \
              and token not in black_list \
              and token.find("\r") == -1 \
              and token.find("\n") == -1 \
              and token.find("\t") == -1 \
              and not (token.isdigit() and len(token) == 1)]
    return ' '.join(tokens)

def generate_queries_lemmas():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in tqdm(root.getchildren()):
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    res.append(Query(task.get("id"), lemmatize(query_text.text), relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res



In [44]:
queries_lemmas = generate_queries_lemmas()

547



547


In [74]:
queries_lemmas[8].query

'инвесткапиталбанк'

In [46]:
lemma_index = Index("lemma_docs", settings_1)

start = time.time()
lemma_index.add_documents("data/json_filtered_tokens_texts")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:02:57.629302


In [47]:
lemma_quality_checker = SearchQualityChecker(queries_lemmas, lemma_index)
lemma_res = lemma_quality_checker.get_results()

In [48]:
lemma_res

(0.21180581149352445,
 0.3860612166406112,
 0.3581352833638026,
 0.2536162197731625)

In [49]:
id_to_pagerank = {}
with open('res/pagerank.txt','r') as f:
    for line in f:
        docId, docURL, rank = line.split()
        id_to_pagerank[int(docId)] = float(rank)

In [50]:
len(id_to_pagerank)


199202

In [51]:
 for doc_name in tqdm(os.listdir("data/json_filtered_tokens_texts")):
        with open(f"data/json_filtered_tokens_texts/{doc_name}", "r+", encoding="utf-8") as inf:
            doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
            doc = json.load(inf)
            try:
                doc["pagerank"] = id_to_pagerank.get(doc_id)
            except:
                pass
            inf.seek(0)        # <--- should reset file position to the beginning.
            json.dump(doc, inf, indent=4, ensure_ascii=False)
            inf.truncate()

In [5]:
settings_with_pagerank = {
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            },
            "pagerank": {
                "type": "rank_feature"
            }
        }
    }
}

In [6]:
pr_index = Index("pagerank_index", settings_with_pagerank)

In [7]:
pr_index.add_documents("data/json_filtered_tokens_texts")


('3 document(s) failed to index.', [{'index': {'_index': 'pagerank_index', '_type': '_doc', '_id': '1204092', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': "failed to parse field [pagerank] of type [rank_feature] in document with id '1204092'. Preview of field's value: '0'", 'caused_by': {'type': 'illegal_argument_exception', 'reason': 'featureValue must be a positive normal float, got: 0.0for feature pagerank on field _feature which is less than the minimum positive normal float: 1.17549435E-38'}}, 'data': {'text': 'english version услуга продукт решение партнер новость компания группа iba контакт поиск главный новость добро пожаловать www kancler смотреть также подписываться наш новость добро пожаловать www kancler пакет прикладной программа ппп канцлер это новое поколение программный продукт наш компания платформа lotus domino notes предназначать создание система электронный документооборот сэд орган государственный управление крупный территориально распред

In [63]:
pr_quality_checker = SearchQualityChecker(queries_lemmas, pr_index)
pr_res = pr_quality_checker.get_results(pagerank_query)

In [64]:
pr_res

(0.20259482033279524,
 0.38557425220134395,
 0.3574040219378429,
 0.24368115581340624)

In [59]:
pr_index.get_doc_by_id(1000039)

{'text': 'сервис компания партнер описание рубрикатор мобильный переводчик sms инфо услуга позволять помощь sms запрос оперативно получать информация различный характер текстовый вид информация sms инфо мобильный доступный 24 час сутки весь территория зона действие сеть velcom ответ запрос приходить оперативно течение несколько секунда получение информация необходимо отправлять sms запрос латиница кириллица номер 511 кроме сформировывать sms запрос помощь sim карта новое образец velcom меню таблица транслитерация соответствие русский буква латинский представлять раздел инструкция sms инфо полный список рубрика sms инфо получать отправлять sms запрос help номер 511 стоимость запрос номер 511 составлять 110 рубль учет ндс налог продажа абонент подробно услуга sms инфо читать раздел инструкция sms инфо внимание информация получать вид mms сообщение помощь услуга mms инфо март поздравлять 23 февраль лотерея ваш лото день св валентина новость спорт деньги день рейтинг погода gismeteo киноне

In [ ]:
settings_titles = {
        'mappings': {
            'properties': {
                'content': {
                    'type': 'text',
                },
                'title': {
                    'type': 'text'
                }
            }
            
        }
    }